In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tarfile
tar = tarfile.open('/content/drive/MyDrive/ML/EnglishFnt.tgz')
tar.extractall('./EnglishFnt')
tar.close()

In [3]:
data = torchvision.datasets.ImageFolder(
    root='/content/EnglishFnt/English/Fnt',
    transform = transforms.Compose([transforms.Resize((48,48)),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, ), (0.5, ))])
)

In [4]:
def load_split(dataset, batch_size, test_split=0.3):
  shuffle_dataset = True
  random_seed = 42
  dataset_size = len(dataset)
  indices = list(range(dataset_size))
  split = int(np.floor(test_split * dataset_size))
  if(shuffle_dataset):
    np.random.seed(random_seed)
    np.random.shuffle(indices)
  train_indices, test_indices = indices[split:], indices[:split]

  testset_size = len(test_indices)
  indices = list(range(testset_size))
  split = int(np.floor(0.5 * testset_size))
  if(shuffle_dataset):
    np.random.seed(random_seed)
    np.random.shuffle(indices)
  val_indices, test_indices = indices[split:], indices[:split]

  # Creating data samplers and loaders
  train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
  val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
  test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)

  train_loader = torch.utils.data.DataLoader(dataset, batch_size,
                                             sampler=train_sampler)
  test_loader = torch.utils.data.DataLoader(dataset, batch_size,
                                             sampler=test_sampler)
  val_loader = torch.utils.data.DataLoader(dataset, batch_size,
                                             sampler=val_sampler)
  
  return train_loader, test_loader, val_loader

In [5]:
batch_size = 36
train_loader, test_loader, val_loader = load_split(data, batch_size, test_split=0.3)

In [7]:
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.conv1 = nn.Conv2d(3,16,3) 
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.conv3 = nn.Conv2d(32, 64, 3)
    self.fc1 = nn.Linear(64 * 9 * 9, 62)

    self.max_pool = nn.MaxPool2d(2,2, ceil_mode=True)
    self.dropout = nn.Dropout(0.2)
    self.conv_bn1 = nn.BatchNorm2d(48,3)
    self.conv_bn2 = nn.BatchNorm2d(16)
    self.conv_bn3 = nn.BatchNorm2d(32)
    self.conv_bn4 = nn.BatchNorm2d(64)

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.max_pool(x)
      x = self.conv_bn2(x)

      x = F.relu(self.conv2(x))
      x = self.max_pool(x)
      x = self.conv_bn3(x)

      x = F.relu(self.conv3(x))
      x = self.max_pool(x)
      x = self.conv_bn4(x)

      x = x.view(-1, 64 * 9 * 9)
      x = self.dropout(x)
      x = self.fc1(x)
      return x

In [9]:
def one_hot(labels, pred_size):
  one_hot_encoded = torch.zeros(len(labels)), pred_size
  y = 0
  for x in labels:
    one_hot_encoded[y][x] = 1
    y += 1
  return one_hot_encoded


In [10]:
class MyLoss(torch.autograd.Function):
  @staticmethod
  def forward(ctx, pred, labels):
    y = one_hot(labels, len(pred[0]))
    y = y.cude()
    ctx.save_for_backward(y, pred)
    loss = - y * torch.log(pred)
    loss = loss.sum()/len(labels)
    return loss

    @staticmethod
    def backward(ctx, grad_output):
      y, pred = ctx.saved_tensors
      grad_input = (-y / pred) - y
      grad_input = grad_input/len(pred)
      return grad_input, grad_output